In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from sbi_lens.gen_dataset.lensing_lognormal_dataset import LensingLogNormalDataset
%pylab inline 
import jax.numpy as jnp
from functools import partial

/gpfslocalsup/pub/anaconda-py3/2022.05/envs/tensorflow-gpu-2.11.0+py3.10.8/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-15 23:49:16.583299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-15 23:49:16.761087: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Pre load and compress dataset 

In [4]:
import haiku as hk
from haiku._src.nets.resnet import ResNet18
import pickle

In [5]:
######## COMPRESSOR ########
print('######## COMPRESSOR ########')

compressor = hk.transform_with_state(
    lambda y : ResNet18(dim)(y, is_training=False)
)

a_file = open('./data/params_compressor/opt_state_resnet_vmim.pkl', "rb") 
opt_state_resnet= pickle.load(a_file)

a_file = open('./data/params_compressor/params_nd_compressor_vmim.pkl', "rb")
parameters_compressor= pickle.load(a_file)

######## COMPRESSOR ########


In [ ]:
from sbi_lens.gen_dataset import lensing_lognormal_dataset
batch_size = 128
dim = 6

ds = tfds.load(
    'LensingLogNormalDataset/year_10_with_noise_score_density_proposal',
    split='train'
)
ds = ds.batch(batch_size)

iter = ds.as_numpy_iterator()

y = []
theta = []
score = []

for example in iter:
  # compress the maps into a summary
  comp_y,_ = compressor.apply(
      parameters_compressor,
      opt_state_resnet,
      None,
      example['simulation']
    )
  y.append(comp_y)
  theta.append(example['theta'])
  score.append(example['score'])
dataset = {
    'theta': np.concatenate(theta, axis=0), 
    'score': np.concatenate(score, axis=0), 
    'y': np.concatenate(y, axis=0), 
}
# saving the dataset
np.savez('LOADED&COMPRESSED_year_10_with_noise_score_density_proposal.npz', dataset)

######## TRAIN ########


2023-07-15 23:49:08.712705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1753 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1a:00.0, compute capability: 7.0


In [6]:
from sbi_lens.gen_dataset import lensing_lognormal_dataset
batch_size = 128
dim = 6

ds = tfds.load(
    'LensingLogNormalDataset/year_10_with_noise_score_density',
    split='train'
)
ds = ds.batch(batch_size)

iter = ds.as_numpy_iterator()

y = []
theta = []
score = []

for example in iter:
  # compress the maps into a summary
  comp_y,_ = compressor.apply(
      parameters_compressor,
      opt_state_resnet,
      None,
      example['simulation']
    )
  y.append(comp_y)
  theta.append(example['theta'])
  score.append(example['score'])
dataset = {
    'theta': np.concatenate(theta, axis=0), 
    'score': np.concatenate(score, axis=0), 
    'y': np.concatenate(y, axis=0), 
}
# saving the dataset
np.savez('LOADED&COMPRESSED_year_10_with_noise_score_density.npz', dataset)

######## TRAIN ########
